# Segmentation in multiple channels (green structure + red puncta)

This notebook loads the provided image, splits channels, segments the big structure in the green channel and the small puncta in the red channel, fills holes, and overlays colored contours on the original image. Intermediate results are saved to disk for inspection.

Setup: imports and output folder.

In [1]:
import os
import numpy as np
from skimage.io import imread, imsave
from skimage.morphology import remove_small_objects
from skimage.segmentation import find_boundaries
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes

import napari_simpleitk_image_processing as nsitk
import napari_segment_blobs_and_things_with_membranes as nsbatwm

results_dir = "results_issue_314"
os.makedirs(results_dir, exist_ok=True)

def save_uint8(path, arr):
    a = np.asarray(arr)
    if a.dtype != np.uint8:
        a = np.clip(a, 0, 255).astype(np.uint8)
    imsave(path, a)
    return path

/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_tools_menu/__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.")
/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_skimage_regionprops/_table.py:7: UserWarning: No Qt bindings could be found
  warnings.warn(str(e))


Load the RGB image from the repository and keep a copy. The file should be in the working directory as `issue_314_image.png`.

In [2]:
image_path = "issue_314_image.png"
image_rgb = imread(image_path)
save_uint8(os.path.join(results_dir, "original.png"), image_rgb.astype(np.uint8))
h, w = image_rgb.shape[0], image_rgb.shape[1]

Split channels and rescale intensity for processing using SimpleITK-based functions. We drop alpha if present and ensure we have R/G/B as uint8. Save channels to disk for inspection.

In [3]:
# Ensure RGB without alpha
if image_rgb.ndim == 2:
    # Grayscale fallback: replicate to 3 channels
    image_rgb = np.stack([image_rgb]*3, axis=-1)
elif image_rgb.shape[-1] > 3:
    image_rgb = image_rgb[..., :3]

R = image_rgb[..., 0]
G = image_rgb[..., 1]
B = image_rgb[..., 2]

# Rescale to 0..255 explicitly
R8 = nsitk.rescale_intensity(R, output_minimum=0, output_maximum=255).astype(np.uint8)
G8 = nsitk.rescale_intensity(G, output_minimum=0, output_maximum=255).astype(np.uint8)

save_uint8(os.path.join(results_dir, "channel_red.png"), R8)
save_uint8(os.path.join(results_dir, "channel_green.png"), G8)

'results_issue_314/channel_green.png'

Segment the large structure in the green channel: Gaussian denoise → Otsu threshold → fill holes → remove small objects → outline. Adjust `sigma` and `min_size` if needed for your data.

In [4]:
# Denoise and threshold
G_smooth = nsbatwm.gaussian_blur(G8, sigma=1.5).astype(np.uint8)
G_bin = nsitk.threshold_otsu(G_smooth)
G_mask = (G_bin > 0)

# Fill holes and clean small speckles
G_mask = binary_fill_holes(G_mask)
G_mask = remove_small_objects(G_mask, min_size=1500)

# Outline
G_outline = find_boundaries(G_mask, mode="outer")

# Save intermediates
save_uint8(os.path.join(results_dir, "green_mask.png"), (G_mask.astype(np.uint8)*255))
save_uint8(os.path.join(results_dir, "green_outline.png"), (G_outline.astype(np.uint8)*255))

/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/skimage/_shared/utils.py:328: UserWarning: results_issue_314/green_mask.png is a low contrast image
  return func(*args, **kwargs)
/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/skimage/_shared/utils.py:328: UserWarning: results_issue_314/green_outline.png is a low contrast image
  return func(*args, **kwargs)


'results_issue_314/green_outline.png'

Segment the small puncta in the red channel: White top-hat for background removal → Gaussian denoise → Otsu threshold → remove small objects → fill small holes → outline. Tune `radius`, `sigma` and `min_size` if needed for your data.

In [5]:
# Background removal and smoothing
R_tophat = nsbatwm.white_tophat(R8, radius=4)
R_smooth = nsbatwm.gaussian_blur(R_tophat, sigma=1.0).astype(np.uint8)

# Threshold
R_bin = nsitk.threshold_otsu(R_smooth)
R_mask = (R_bin > 0)

# Clean small noise and fill small holes within spots
R_mask = remove_small_objects(R_mask, min_size=20)
R_mask = binary_fill_holes(R_mask)

# Outline
R_outline = find_boundaries(R_mask, mode="outer")

# Save intermediates
save_uint8(os.path.join(results_dir, "red_mask.png"), (R_mask.astype(np.uint8)*255))
save_uint8(os.path.join(results_dir, "red_outline.png"), (R_outline.astype(np.uint8)*255))

'results_issue_314/red_outline.png'

Create overlays: original image with contours for green and red in different colors using matplotlib. Figures are saved to disk for later inspection and reuse.

In [6]:
# Helper for overlay plotting
def save_overlay(path, show_green=True, show_red=True):
    plt.figure(figsize=(6, 6))
    plt.imshow(image_rgb)
    if show_green:
        plt.contour(G_mask.astype(float), levels=[0.5], colors=['lime'], linewidths=1.8)
    if show_red:
        plt.contour(R_mask.astype(float), levels=[0.5], colors=['magenta'], linewidths=1.2)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(path, dpi=200, bbox_inches='tight')
    plt.close()

save_overlay(os.path.join(results_dir, "overlay_green.png"), show_green=True, show_red=False)
save_overlay(os.path.join(results_dir, "overlay_red.png"), show_green=False, show_red=True)
save_overlay(os.path.join(results_dir, "overlay_both.png"), show_green=True, show_red=True)

Optionally, save binary masks as NumPy arrays for downstream analysis in other scripts or notebooks (e.g., measurement, co-localization).

In [7]:
np.save(os.path.join(results_dir, "green_mask.npy"), G_mask.astype(np.uint8))
np.save(os.path.join(results_dir, "red_mask.npy"), R_mask.astype(np.uint8))